Step 0. Unzip enron1.zip into the current directory.

Step 1. Traverse the dataset and create a Pandas dataframe. This is already done for you and should run without any errors. You should recognize Pandas from task 1.

In [13]:
import pandas as pd
import os

def read_spam():
    category = 'spam'
    directory = './enron1/spam'
    return read_category(category, directory)

def read_ham():
    category = 'ham'
    directory = './enron1/ham'
    return read_category(category, directory)

def read_category(category, directory):
    emails = []
    for filename in os.listdir(directory):
        if not filename.endswith(".txt"):
            continue
        with open(os.path.join(directory, filename), 'r') as fp:
            try:
                content = fp.read()
                emails.append({'name': filename, 'content': content, 'category': category})
            except:
                print(f'skipped {filename}')
    return emails

ham = read_ham()
spam = read_spam()

df = pd.DataFrame.from_records(ham)

df = pd.concat([df_ham, df_spam], ignore_index=True)


skipped 2649.2004-10-27.GP.spam.txt
skipped 0754.2004-04-01.GP.spam.txt
skipped 2042.2004-08-30.GP.spam.txt
skipped 3304.2004-12-26.GP.spam.txt
skipped 4142.2005-03-31.GP.spam.txt
skipped 3364.2005-01-01.GP.spam.txt
skipped 4201.2005-04-05.GP.spam.txt
skipped 2140.2004-09-13.GP.spam.txt
skipped 2248.2004-09-23.GP.spam.txt
skipped 4350.2005-04-23.GP.spam.txt
skipped 4566.2005-05-24.GP.spam.txt
skipped 2526.2004-10-17.GP.spam.txt
skipped 1414.2004-06-24.GP.spam.txt
skipped 2698.2004-10-31.GP.spam.txt
skipped 5105.2005-08-31.GP.spam.txt


Step 2. Data cleaning is a critical part of machine learning. You and I can recognize that 'Hello' and 'hello' are the same word but a machine does not know this a priori. Therefore, we can 'help' the machine by conducting such normalization steps for it. Write a function `preprocessor` that takes in a string and replaces all non alphabet characters with a space and then lowercases the result.

In [14]:
import re

def preprocessor(e):
    return re.sub('[^A-Za-z]', ' ', e).lower()

Step 3. We will now train the machine learning model. All the functions that you will need are imported for you. The instructions explain how the work and hint at which functions to use. You will likely need to refer to the scikit learn documentation to see how exactly to invoke the functions. It will be handy to keep that tab open.

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# The CountVectorizer converts a text sample into a vector (think of it as an array of floats).
# Each entry in the vector corresponds to a single word and the value is the number of times the word appeared.
# Instantiate a CountVectorizer. Make sure to include the preprocessor you previously wrote in the constructor.
# TODO


# Use train_test_split to split the dataset into a train dataset and a test dataset.
# The machine learning model learns from the train dataset.
# Then the trained model is tested on the test dataset to see if it actually learned anything.
# If it just memorized for example, then it would have a low accuracy on the test dataset and a high accuracy on the train dataset.
# TODO


# Use the vectorizer to transform the existing dataset into a form in which the model can learn from.
# Remember that simple machine learning models operate on numbers, which the CountVectorizer conveniently helped us do.
# TODO


# Use the LogisticRegression model to fit to the train dataset.
# You may remember y = mx + b and Linear Regression from high school. Here, we fitted a scatter plot to a line.
# Logistic Regression is another form of regression. 
# However, Logistic Regression helps us determine if a point should be in category A or B, which is a perfect fit.
# TODO


# Validate that the model has learned something.
# Recall the model operates on vectors. First transform the test set using the vectorizer. 
# Then generate the predictions.
# TODO


# We now want to see how we have done. We will be using three functions.
# `accuracy_score` tells us how well we have done. 
# 90% means that every 9 of 10 entries from the test dataset were predicted accurately.
# The `confusion_matrix` is a 2x2 matrix that gives us more insight.
# The top left shows us how many ham emails were predicted to be ham (that's good!).
# The bottom right shows us how many spam emails were predicted to be spam (that's good!).
# The other two quadrants tell us the misclassifications.
# Finally, the `classification_report` gives us detailed statistics which you may have seen in a statistics class.
# TODO


vectorizer = CountVectorizer(preprocessor=preprocessor)
X_train, X_test, y_train, y_test = train_test_split(df['content'],df['category'], test_size=0.2, random_state=42)
X_train_df = vectorizer.fit_transform(X_train)

model = LogisticRegression()
model.fit(X_train_df, y_train)

X_test_df = vectorizer.transform(X_test)
y_pred = model.predict(X_test_df)

print(f'Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}\n')
print(f'Detailed Statistics: \n{classification_report(y_test, y_pred)}')


Confusion Matrix: 
[[693  14]
 [ 11 314]]

Detailed Statistics: 
              precision    recall  f1-score   support

         ham       0.98      0.98      0.98       707
        spam       0.96      0.97      0.96       325

    accuracy                           0.98      1032
   macro avg       0.97      0.97      0.97      1032
weighted avg       0.98      0.98      0.98      1032



Step 4.

In [16]:
# Let's see which features (aka columns) the vectorizer created. 
# They should be all the words that were contained in the training dataset.
# TODO


# You may be wondering what a machine learning model is tangibly. It is just a collection of numbers. 
# You can access these numbers known as "coefficients" from the coef_ property of the model
# We will be looking at coef_[0] which represents the importance of each feature.
# What does importance mean in this context?
# Some words are more important than others for the model.
# It's nothing personal, just that spam emails tend to contain some words more frequently.
# This indicates to the model that having that word would make a new email more likely to be spam.
# TODO


# Iterate over importance and find the top 10 positive features with the largest magnitude.
# Similarly, find the top 10 negative features with the largest magnitude.
# Positive features correspond to spam. Negative features correspond to ham.
# You will see that `http` is the strongest feature that corresponds to spam emails. 
# It makes sense. Spam emails often want you to click on a link.
# TODO

features = vectorizer.get_feature_names_out()
importance = model.coef_[0]

indexed_importance = list(enumerate(importance))
sorted_importance = sorted(indexed_importance, key=lambda x: abs(x[1]), reverse=True)

top_positive = [(index, importance) for index, importance in sorted_importance if importance > 0][:10]
top_negative = [(index, importance) for index, importance in sorted_importance if importance < 0][:10]

df_top_positive = pd.DataFrame(top_positive, columns=['Feature Index', 'Importance'])
df_top_negative = pd.DataFrame(top_negative, columns=['Feature Index', 'Importance'])

df_top_positive['Feature'] = [features[index] for index, _ in top_positive]
df_top_negative['Feature'] = [features[index] for index, _ in top_negative]

print('Top 10 Positive Features')
print(df_top_positive)

print('\nTop 10 Negative Features')
print(df_top_negative)

Top 10 Positive Features
   Feature Index  Importance  Feature
0          17754    0.894404     http
1          17019    0.814249    hello
2          30340    0.780498  removed
3          28501    0.773066   prices
4          25157    0.757636       no
5          24007    0.745125    money
6          17082    0.706949     here
7          25961    0.659537     only
8          26564    0.647642     pain
9          24107    0.635354     more

Top 10 Negative Features
   Feature Index  Importance   Feature
0           2558   -1.691388  attached
1          12606   -1.555386     enron
2           9459   -1.429007     daren
3          35650   -1.332251    thanks
4          11028   -1.285912       doc
5           9641   -1.237547      deal
6          23381   -1.115141     meter
7          17678   -1.108359       hpl
8          24834   -1.000763      neon
9          39776   -0.951037       xls


Submission
1. Upload the jupyter notebook to Forage.

All Done!